# Projet maison n° 3

In [1]:
# imports
import pandas as pd

## 1. US baby names

On va s'intéresser au dataset **National data** de la SSA : https://www.ssa.gov/oact/babynames/limits.html

1. Télécharger le dataset des prénoms US : https://www.ssa.gov/oact/babynames/names.zip

2. Implémenter une fonction Python qui produit un unique DataFrame avec tous les fichiers en utilisant pandas  (par ex. fonction \"concat\" ou méthode \"append\"), pas de bash :)

Ordre et noms des colonnes : 'year', 'name', 'gender', 'births'

Le DataFrame doit être trié selon l'année croissante puis selon l'ordre défini par les différents fichiers (voir la documentation ci-dessus).

In [2]:
# 1) implémentation avec glob + append
import glob

def df_names_us():
    df = pd.DataFrame()
    files = glob.glob('names/*.txt')
    files.sort()
    for filename in files:
        csv = pd.read_csv(filename,
                          names=['name', 'gender', 'births'])
        csv['year'] = int(filename[-8:-4]) # yobAAAA.txt
        df = df.append(csv, ignore_index=True)
    df = df[['year', 'name', 'gender', 'births']]
    return df

In [3]:
%%time
df_us = df_names_us()
df_us.shape

Wall time: 7.59 s


(2020863, 4)

In [4]:
# head
df_us.head()

,year,name,gender,births
0,1880,Mary,F,7065
1,1880,Anna,F,2604
2,1880,Emma,F,2003
3,1880,Elizabeth,F,1939
4,1880,Minnie,F,1746


In [5]:
# 2) implémentation avec range + concat
def df_names_us():
    dfs = []
    for year in range(1880, 2021):
        csv = pd.read_csv(f'names/yob{year}.txt',
                          header=None,
                          names=['name', 'gender', 'births'])
        csv['year'] = year
        dfs.append(csv)
    df = pd.concat(dfs, ignore_index=True)
    df = df[['year', 'name', 'gender', 'births']]
    return df

In [6]:
%%time
df_us = df_names_us()
df_us.shape

Wall time: 2.05 s


(2020863, 4)

In [7]:
# head
df_us.head()

,year,name,gender,births
0,1880,Mary,F,7065
1,1880,Anna,F,2604
2,1880,Emma,F,2003
3,1880,Elizabeth,F,1939
4,1880,Minnie,F,1746


## 2. Prénoms français

On va s'intéresser au dataset **Fichiers France hors Mayotte** de l'INSEE :  https://www.insee.fr/fr/statistiques/2540004/

L'idée est de charger les données et ensuite de les conformer au DataFrame des prénoms US. Ainsi, toute manipulation sur le DataFrame des prénoms US pourra être directement réutilisée avec le DataFrame des prénoms français.
 
1. Télécharger le dataset des prénoms français : https://www.insee.fr/fr/statistiques/fichier/2540004/nat2019_csv.zip


Lire la documentation, ça peut être utile...
 
2. Implémenter une fonction Python qui produit un DataFrame avec les prénoms français en prenant le DataFrame des prénoms US comme modèle :
 
 - Même ordre et mêmes noms des colonnes : year, name, gender, births
 - Mêmes dtypes pour les colonnes
 - Mêmes valeurs pour la colonne 'gender'
 - Seuls les prénoms de 2 caractères et plus sont conservés
 - La casse des prénoms doit être identique : initiales en majuscule, autres lettres en minuscule
 - Les lignes avec des données inutilisables doivent être supprimées
 - Les données sont triées à l'identique : années (↑), puis gender (↑), puis births (↓) et enfin name (↑)
 - L'index du DataFrame doit aller de 0 à N-1

In [8]:
def df_names_fr():
    # genres
    d = {'1': 'M', '2': 'F'}
    # read_csv
    df = pd.read_csv('nat2020_csv.zip',
                      sep=';',
                      header=0,
                      names=['gender', 'name', 'year', 'births'],
                      converters={
                          'gender': d.get,
                          'name': str.title
                      })
    # clean
    df = df.loc[(df['name'].str.len() > 1)
                & (df['year'] != 'XXXX')
                & ~df['name'].str.startswith('_')]
    # types
    df['year'] = df['year'].astype(int)
    # ordre colonnes
    df = df[['year', 'name', 'gender', 'births']]
    # tri
    df = df.sort_values(['year', 'gender', 'births', 'name'],
                   ascending=[True, True, False, True])
    df = df.reset_index(drop=True)
    
    return df

In [9]:
%%time
df_fr = df_names_fr()
df_fr.shape

Wall time: 2 s


(630407, 4)

In [10]:
# head
df_fr.head()

,year,name,gender,births
0,1900,Marie,F,48713
1,1900,Jeanne,F,13981
2,1900,Marguerite,F,8058
3,1900,Germaine,F,6981
4,1900,Louise,F,6696


In [11]:
df_fr.dtypes

year       int32
name      object
gender    object
births     int64
dtype: object

In [12]:
# prénom NA
df_fr.loc[df_fr['name']=='Na']

,year,name,gender,births
404391,2003,Na,F,3


In [13]:
# impact du converters / name
d = {'1': 'M', '2': 'F'}

df = pd.read_csv('nat2020_csv.zip',
                  sep=';',
                  header=0,
                  names=['gender', 'name', 'year', 'births'],)
                  #converters={
                  #    'gender': d.get,
                  #    'name': str.title
                  #})

df['name'].isnull().sum()

2

In [14]:
df.isnull().any()

gender    False
name       True
year      False
births    False
dtype: bool

In [16]:
df.loc[df['name'].isnull()]

,gender,name,year,births
563597,2,NaN,2003,3
563598,2,NaN,XXXX,29


In [17]:
# capitalize
'JEAN-MARIE'.capitalize()

'Jean-marie'

In [18]:
# title
'JEAN-MARIE'.title()

'Jean-Marie'

## 3. Taux de change

On va s'intéresser au dataset des cours des devises de la Banque de France :  http://webstat.banque-france.fr/fr/browseBox.do?node=5385566

L'idée est de charger les données, de les nettoyer et de pouvoir accéder aux cours de certaines devises à partir de leur code ISO3.
 
1. Télécharger le dataset des taux de change : http://webstat.banque-france.fr/fr/downloadFile.do?id=5385698&exportType=csv


2. Implémenter une fonction qui produit un DataFrame avec les taux de change par date pour une liste de codes ISO3 de devises passée en argument. L'index du DataFrame doit correspondre aux dates (voir la fonction pd.to_datetime() avec le format '%d/%m/%Y'). Les colonnes du DataFrame doivent correspondre aux devises.

In [19]:
def df_taux_change(devises):
    df = pd.read_csv("Webstat_Export.csv",
                     sep=";",
                     na_values='-',
                     decimal=',',
                     skiprows=[0, 1, 3, 4, 5],  # le skiprows permet à l'option "decimal" de fonctionner
                     converters={0: lambda x: pd.to_datetime(x, format='%d/%m/%Y', errors='ignore')})

    # extraction des codes monnaies
    cols = pd.Series(df.columns.tolist()).str.extract('\(([A-Z]{3})\)', expand=True)
    cols.iloc[0] = 'Date'
    df.columns = cols[0]

    # selection des devises
    df = df[['Date'] + devises]

    # drop na
    df = df.dropna()

    # set index
    df = df.set_index('Date')
    
    return df

In [20]:
%%time
df_tx = df_taux_change(['CHF', 'GBP', 'USD'])
df_tx.shape

Wall time: 1.18 s


(5656, 3)

In [21]:
# head
df_tx

,CHF,GBP,USD
Date,,,
2021-02-04,1.0818,0.87693,1.1996
2021-02-03,1.0804,0.88145,1.2017
2021-02-02,1.0808,0.88075,1.2044
2021-02-01,1.0816,0.88200,1.2084
2021-01-29,1.0798,0.88383,1.2136
...,...,...,...
1999-01-08,1.6138,0.70940,1.1659
1999-01-07,1.6165,0.70585,1.1632
1999-01-06,1.6116,0.70760,1.1743


In [22]:
df_tx.index

DatetimeIndex(['2021-02-04', '2021-02-03', '2021-02-02', '2021-02-01',
               '2021-01-29', '2021-01-28', '2021-01-27', '2021-01-26',
               '2021-01-25', '2021-01-22',
               ...
               '1999-01-15', '1999-01-14', '1999-01-13', '1999-01-12',
               '1999-01-11', '1999-01-08', '1999-01-07', '1999-01-06',
               '1999-01-05', '1999-01-04'],
              dtype='datetime64[ns]', name='Date', length=5656, freq=None)

### Tests

In [23]:
import unittest

class Lesson4Tests(unittest.TestCase):
    def test_df_names_us(self):
        df = df_names_us()
        # colonnes
        self.assertEqual(list(df.columns), ['year', 'name', 'gender', 'births'])
        # lignes
        self.assertEqual(len(df), 2020863)
        # index
        self.assertTrue(isinstance(df.index, pd.core.indexes.range.RangeIndex))
        # test NaN
        self.assertTrue(df.loc[df.isnull().any(axis=1)].empty)
        
    def test_df_names_fr(self):
        df = df_names_fr()
        # colonnes
        self.assertEqual(list(df.columns), ['year', 'name', 'gender', 'births'])
        # lignes
        self.assertEqual(len(df), 630407)
        # index
        self.assertTrue(isinstance(df.index, pd.core.indexes.range.RangeIndex))
        # test names
        self.assertTrue(df.loc[df['name'].str.contains('^[A-Z]+(?:-[A-Z]+)?$')].empty)
        # test gender
        self.assertEqual(len(df), len(df.loc[df['gender']=='F']) + len(df.loc[df['gender']=='M']))
        # test NaN
        self.assertTrue(df.loc[df.isnull().any(axis=1)].empty)

    def test_df_taux_change(self):
        df = df_taux_change(['CHF', 'GBP', 'USD'])
        # colonnes
        self.assertEqual(list(df.columns), ['CHF', 'GBP', 'USD'])
        # index
        self.assertTrue(isinstance(df.index, pd.core.indexes.datetimes.DatetimeIndex))
        # types taux
        self.assertTrue((df.dtypes == 'float').all())
        # test NaN
        self.assertTrue(df.loc[df.isnull().any(axis=1)].empty)

In [24]:
# run tests
def run_tests():
    test_suite = unittest.makeSuite(Lesson4Tests)
    runner = unittest.TextTestRunner(verbosity=2)
    runner.run(test_suite)

In [25]:
# run tests

run_tests()

test_df_names_fr (__main__.Lesson4Tests) ... ok
test_df_names_us (__main__.Lesson4Tests) ... ok
test_df_taux_change (__main__.Lesson4Tests) ... ok

----------------------------------------------------------------------
Ran 3 tests in 6.292s

OK


### Session 5 - Un peu de Data Science...

**Question n° 1**

Pourquoi le value_counts() du "gender" donne-t-il un tel écart entre F et M ?

In [ ]:
# value_counts du gender

df_us['gender'].value_counts()

In [ ]:
# value_counts du gender

df_fr['gender'].value_counts()

In [ ]:
# pct_change

df_us['gender'].value_counts().pct_change()

In [ ]:
# pct_change

df_fr['gender'].value_counts().pct_change()

**Question n° 2**

Pourquoi le value_counts() du "name" donne-t-il ce résultat ?

In [ ]:
# value_counts du gender

df_us['name'].value_counts().head(16)

In [ ]:
# value_counts du gender

df_fr['name'].value_counts().head(16)

**Question n° 3**

Pourquoi le value_counts() du "year" donne-t-il ce résultat ?

In [ ]:
# value_counts du gender

df_us['year'].value_counts().head(16)

In [ ]:
# value_counts du gender

df_fr['year'].value_counts().head(16)

**Exercice n° 1**

Donnez le prénom qui a été le plus donné lors d'une année.

**Exercice n° 2**

Donnez la liste des prénoms qui contiennent dans l'ordre a, e, i, o et u (US) ou bien a, e, i et o (FR).

### Méthodes de reshaping (1)

#### pivot_table

La méthode pivot_table() prend en argument :
- values : valeurs qui doivent être agrégées selon les modalités de la colonne passée en "index" et de la colonne passée en "columns"
- index : colonne(s) dont les valeurs vont servir d'index à la table pivot
- columns : colonne(s) dont les valeurs vont servir de colonnes à la table pivot
- aggfunc : fonction d'aggrégation des values, par défaut 'mean', 'median', 'min', 'max', 'count', 'sum', 'nunique', et n'importe quelle lambda ou liste de fonctions.

On obtient NaN s'il n'y a pas d'occurence croisée.

Excel = tableau croisé dynamique.

In [ ]:
# exemple
df = pd.DataFrame([{'A': 1,'B': 1, 'C': 1},
                   {'A': 1,'B': 1, 'C': 2},
                   {'A': 1,'B': 2, 'C': -1},
                   {'A': 2,'B': 1, 'C': 4},
                   {'A': 2,'B': 1, 'C': 5},
                  ])

df

In [ ]:
# exemple
df.pivot_table(values='C',
              index='A',
              columns='B',
              aggfunc='mean')

**Exercice n° 3**

Calculez une table pivot avec le nombre total de naissances par année et par genre.

**Exercice n° 4**

Calculez une table pivot avec la diversité des prénoms (nombre de prénoms distincts) par année et par genre.

#### crosstab

crosstab() est une fonction de reshaping qui prend 2 colonnes d'un DataFrame en argument et produit le décompte croisé des occurrence.

On obtient 0 s'il n'y a pas d'occurence croisée.

In [ ]:
# exemple
pd.crosstab(df['A'], df['B'])

In [ ]:
# initial
df_us['initial'] = df_us['name'].str[0].str.upper()
# terminal
df_us['terminal'] = df_us['name'].str[-1].str.upper()

df_us.head()

In [ ]:
# crosstab
pd.crosstab(df_us['initial'], df_us['terminal'], normalize=True, margins=True)*100

In [ ]:
# pivot_table
df_us.pivot_table(values='name',
                  index='initial',
                  columns='terminal',
                  aggfunc='nunique').fillna(0).astype(int)

In [ ]:
# Z x Z
df_us.loc[(df_us['initial'] == 'Z') & (df_us['terminal'] == 'Z')]

In [ ]:
# Z x Z
df_us.loc[df_us['name'].str.startswith('Z') & df_us['name'].str.endswith('z')]